In [42]:
import pandas as pd
df = pd.read_csv('ques.csv')
df

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...,...
14995,14995,14995,29932,29933,What strikes first time visitors as special or...,What strikes first time visitors as special or...,0
14996,14996,14996,29934,29935,I want to be a poet what should I do to get my...,I am a budding poet and I want to get publishe...,1
14997,14997,14997,29936,29937,What is borderline personality disorder?,What is a borderline personality?,1
14998,14998,14998,29938,29939,Are there more colors in a rainbow than the se...,Are there more chickens than people in the world?,0


In [43]:
print(df.isnull().sum())

Unnamed: 0      0
id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64


In [44]:
df.dropna(inplace=True)
print(df.isnull().sum())

Unnamed: 0      0
id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    15000 non-null  int64 
 1   id            15000 non-null  int64 
 2   qid1          15000 non-null  int64 
 3   qid2          15000 non-null  int64 
 4   question1     15000 non-null  object
 5   question2     15000 non-null  object
 6   is_duplicate  15000 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 820.4+ KB


In [46]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the dataset
df = df

# Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    explicit_words = ['sex']
    tokens = word_tokenize(text.lower())
    # Filter out explicit words and stopwords
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words and token not in explicit_words]

    return ' '.join(tokens)


df['question1'] = df['question1'].apply(preprocess_text)
df['question2'] = df['question2'].apply(preprocess_text)

# Split the dataset into training and testing sets
X = df[['question1', 'question2']]
y = df['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train.apply(lambda x: ' '.join(x), axis=1))
X_test_vectorized = vectorizer.transform(X_test.apply(lambda x: ' '.join(x), axis=1))

# Train a classifier (Logistic Regression in this example)
classifier = LogisticRegression()
classifier.fit(X_train_vectorized, y_train)

# Make predictions
predictions = classifier.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

cross_val_scores = cross_val_score(classifier, X_train_vectorized, y_train, cv=5)
print("Cross-Validation Scores:", cross_val_scores)

confusion_mat = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(confusion_mat)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cheth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cheth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cheth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Accuracy: 0.7053333333333334
Cross-Validation Scores: [0.70666667 0.69428571 0.7047619  0.69380952 0.72095238]
Confusion Matrix:
[[2533  357]
 [ 969  641]]


In [47]:
import pickle

# Train a classifier (Logistic Regression in this example)
classifier = LogisticRegression()
classifier.fit(X_train_vectorized, y_train)

# Save the trained model
pickle.dump(classifier, open('trained_model.pkl', 'wb'))

# Save the vectorizer
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))


In [48]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle

# Load the new dataset
new_data = pd.read_csv('pred_ques.csv')

# Preprocess the new data
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    explicit_words = ['sex']
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Filter out explicit words and stopwords
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words and token not in explicit_words]

    return ' '.join(tokens)

new_data['question1'] = new_data['question1'].apply(preprocess_text)
new_data['question2'] = new_data['question2'].apply(preprocess_text)

# Load the saved model
loaded_model = pickle.load(open('trained_model.pkl', 'rb'))

# Load the saved vectorizer
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))

# Vectorize the new data using the loaded vectorizer
X_new = vectorizer.transform(new_data['question1'])

# Make predictions on the new data
predictions = loaded_model.predict(X_new)

# Analyze and use the predictions
for i, prediction in enumerate(predictions):
    print(f"Question {i+1}: {new_data['question1'][i]}")
    print(f"Prediction: {'Duplicate' if prediction == 1 else 'Not Duplicate'}")
    print()

# Additional steps as per your requirements


Question 1: india bangladesh like nepal bhutan
Prediction: Not Duplicate

Question 2: difference school life college life
Prediction: Duplicate

Question 3: forgot gmail username access recovery phone number gmail account
Prediction: Duplicate

Question 4: average sale dollar average number sale fast casual restaurant california per day week month etc
Prediction: Not Duplicate

Question 5: fall asleep fast tired
Prediction: Not Duplicate

Question 6: start information security
Prediction: Not Duplicate

Question 7: watch netflix laptop
Prediction: Not Duplicate

Question 8: kind psychological problem india
Prediction: Not Duplicate

Question 9: best night place hang stuttgart germany
Prediction: Not Duplicate

Question 10: important cause effect fall roman empire
Prediction: Not Duplicate

Question 11: reduce belly chest fat
Prediction: Not Duplicate

Question 12: zip code alberta canada
Prediction: Not Duplicate

Question 13: difference radiation wavelength
Prediction: Not Duplicate

